In [ ]:
spark

In [ ]:
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
import pyspark.sql.functions as psf
import pandas as pd
import numpy as np
from pyspark.sql.types import *

In [ ]:
CHI_data=spark.read.parquet("/user/datascience/customer_happiness/dump/binary/event_date=2019-02-28")

In [ ]:
CHI_data.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- happiness: integer (nullable = true)
 |-- probability: double (nullable = true)
 |-- circle: integer (nullable = true)



In [ ]:
#REC_data=spark.sql("select * from edw_agg.fact_rec_base_dly where event_date='2019-03-15' ")

In [ ]:
feature_table=spark.sql("select msisdn,aon,aon_curr_handset,arpu, data_volume_4g,handset_4g_flag,usim_4g_user \
        from edw_mart.fact_feature_engg_new where event_date='2019-03-15'")

In [ ]:
feature_table.show()

+----------+---+----------------+---------+--------------+---------------+------------+
|    msisdn|aon|aon_curr_handset|     arpu|data_volume_4g|handset_4g_flag|usim_4g_user|
+----------+---+----------------+---------+--------------+---------------+------------+
|6203699043|140|              43| 7.531818|          0.00|              1|           1|
|6230440624| 30|              30|62.071032|       4931.59|              1|           1|
|6232090084|162|              29|51.310464|          0.00|              0|           1|
|6232449130| 97|              36| 0.000000|       2074.94|              1|           1|
|6232538558| 42|              42|48.330419|          0.00|              0|           1|
|6232713837|116|               6|37.927292|          0.00|              0|           1|
|6232782353|142|             135| 0.000000|          0.00|              0|           1|
|6232931082| 85|              70| 0.000000|          0.00|              1|           1|
|6261500488|231|               5

In [ ]:
uci_dump=spark.read.parquet("/user/B0204116/customer360/uci/dump/event_date=2019-02-15")

In [ ]:
uci_dump.printSchema()

root
 |-- subid: string (nullable = true)
 |-- uci_id: long (nullable = true)
 |-- lob: string (nullable = true)
 |-- num_subscriptions: long (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- arpu: string (nullable = true)
 |-- arpa: double (nullable = true)
 |-- circle: string (nullable = true)



In [ ]:
uci_dump.select("arpa").describe().show()

+-------+------------------+
|summary|              arpa|
+-------+------------------+
|  count|         380181082|
|   mean|134.31577831236524|
| stddev|316.16184452373676|
|    min|          -30546.5|
|    max|132745.83333333334|
+-------+------------------+



In [ ]:
uci_dump.stat.approxQuantile("arpa",[0.2],0.25)

[-30546.5]

In [ ]:
uci_dump.stat.approxQuantile("arpa",[0.6],0.25)

[107165.89666666673]

In [ ]:
uci_dump.stat.approxQuantile("arpa",[0.7],0.25)

[107165.89666666673]

In [ ]:
uci_dump_flt=uci_dump.select("subid","arpu","arpa")

In [ ]:
arpa_data=CHI_data.join(uci_dump_flt,CHI_data.msisdn==uci_dump_flt.subid, how="left")

In [ ]:
arpa_data.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- happiness: integer (nullable = true)
 |-- probability: double (nullable = true)
 |-- circle: integer (nullable = true)
 |-- subid: string (nullable = true)
 |-- arpu: string (nullable = true)
 |-- arpa: double (nullable = true)



In [ ]:
arpa_data = arpa_data.withColumn("arpa",arpa_data["arpa"].cast(IntegerType()))

In [ ]:
arpa_data = arpa_data.withColumn("arpu",arpa_data["arpu"].cast(IntegerType()))

In [ ]:
arpa_data=arpa_data.fillna(0)

In [ ]:
#ARPA_t6=arpa_data.groupBy("circle","happiness").pivot("happiness").agg(
 #   psf.mean("arpa").alias("ARPA"))

In [ ]:
#ARPA_t6.show()

In [ ]:
#ARPA_t6_final=ARPA_t6.select("circle","0","1")

In [ ]:
#ARPA_t6_final.show()

In [ ]:
def median(values_list):
    med = np.median(values_list)
    return float(med)
udf_median = psf.udf(median, FloatType())

In [ ]:
# ARPA_table_6=arpa_data.groupBy("circle","happiness").pivot("happiness").agg(
#     udf_median(psf.collect_list(psf.col('arpa'))).alias("Median_ARPA"))

In [ ]:
ARPA_table_6=arpa_data.groupBy("circle","happiness").agg(
    udf_median(psf.collect_list(psf.col('arpa'))).alias("Median_ARPA"))

In [ ]:
ARPU_table_6=arpa_data.groupBy("circle","happiness").agg(
    udf_median(psf.collect_list(psf.col('arpu'))).alias("Median_ARPU"))

In [ ]:
ARPA_table_6.show()

+------+---------+-----------+
|circle|happiness|Median_ARPA|
+------+---------+-----------+
|   118|        1|       30.0|
|   105|        1|      133.0|
|   108|        1|      118.0|
|   119|        0|       19.0|
|   104|        0|       32.0|
|   123|        0|       32.0|
|   117|        1|      113.0|
|   120|        1|       90.0|
|   121|        1|      108.0|
|   112|        1|      100.0|
|   107|        0|       50.0|
|   109|        1|      108.0|
|   122|        0|       17.0|
|   111|        1|       72.0|
|   118|        0|       61.0|
|   101|        0|       55.0|
|   111|        0|       10.0|
|   108|        0|       16.0|
|   120|        0|       41.0|
|   104|        1|      112.0|
+------+---------+-----------+
only showing top 20 rows



In [ ]:
ARPU_table_6.show()

+------+---------+-----------+
|circle|happiness|Median_ARPU|
+------+---------+-----------+
|   108|        1|      108.0|
|   118|        1|       10.0|
|   105|        1|      110.0|
|   119|        0|        9.0|
|   104|        0|       12.0|
|   123|        0|        9.0|
|   117|        1|      105.0|
|   120|        1|       62.0|
|   121|        1|       88.0|
|   112|        1|       87.0|
|   107|        0|       21.0|
|   109|        1|       91.0|
|   122|        0|       15.0|
|   111|        1|       48.0|
|   101|        0|       18.0|
|   118|        0|       41.0|
|   111|        0|        1.0|
|   120|        0|       18.0|
|   108|        0|        3.0|
|   104|        1|       96.0|
+------+---------+-----------+
only showing top 20 rows



In [ ]:
#ARPA_table_6=ARPA_table_6.select("circle","0","1")

In [ ]:
ARPA_table_6=ARPA_table_6.orderBy("circle")

In [ ]:
ARPU_table_6=ARPU_table_6.orderBy("circle")

In [ ]:
#ARPA_table_6.count()

46

In [ ]:
#ARPA_table_6_pd=ARPA_table_6.toPandas()

In [ ]:
ARPA_data=ARPA_table_6.groupBy('circle').pivot('happiness').agg(psf.max('Median_ARPA').alias('Median_ARPA'))

In [ ]:
ARPU_data=ARPU_table_6.groupBy('circle').pivot('happiness').agg(psf.max('Median_ARPU').alias('Median_ARPU'))

In [ ]:
ARPA_data=ARPA_data.withColumnRenamed("0","Unhappy_customer_ARPA")

In [ ]:
ARPA_data=ARPA_data.withColumnRenamed("1","Happy_customer_ARPA")

In [ ]:
ARPU_data=ARPU_data.withColumnRenamed("0","Unhappy_customer_ARPU")

In [ ]:
ARPU_data=ARPU_data.withColumnRenamed("1","Happy_customer_ARPU")

In [ ]:
ARPA_data.toPandas().to_csv("ARPA_customer_happiness_wise.csv",index=False,header=True)

In [ ]:
ARPU_data.toPandas().to_csv("ARPU_customer_happiness_wise.csv",index=False,header=True)

In [ ]:
AON_AOD_data=CHI_data.join(feature_table,on="msisdn",how="left")

In [ ]:
AON_AOD_data.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- happiness: integer (nullable = true)
 |-- probability: double (nullable = true)
 |-- circle: integer (nullable = true)
 |-- aon: short (nullable = true)
 |-- aon_curr_handset: integer (nullable = true)
 |-- arpu: decimal(18,6) (nullable = true)
 |-- data_volume_4g: decimal(18,2) (nullable = true)
 |-- handset_4g_flag: short (nullable = true)
 |-- usim_4g_user: string (nullable = true)



In [ ]:
AON_AOD_data = AON_AOD_data.withColumn("aon_curr_handset",AON_AOD_data["aon_curr_handset"].cast(IntegerType()))

In [ ]:
AON_AOD_data = AON_AOD_data.withColumn("aon",AON_AOD_data["aon"].cast(IntegerType()))

In [ ]:
AON_AOD_data = AON_AOD_data.withColumn("usim_4g_user",AON_AOD_data["usim_4g_user"].cast(IntegerType()))

In [ ]:
AON_AOD_data=AON_AOD_data.fillna(0)

In [ ]:
Users_4g=AON_AOD_data.where("handset_4g_flag = 1 and usim_4g_user = 1 ")

In [ ]:
Users_4g_data=Users_4g.groupBy("circle","happiness").count()

In [ ]:
Users_4g_data.show()

+------+---------+-------+
|circle|happiness|  count|
+------+---------+-------+
|   118|        1|3149410|
|   105|        1|6441014|
|   108|        1| 870751|
|   119|        0|5904750|
|   104|        0|7719671|
|   123|        0|1504204|
|   117|        1| 976733|
|   120|        1|1442709|
|   121|        1|3378396|
|   112|        1|2698799|
|   107|        0|2849162|
|   109|        1|2638923|
|   122|        0|1565366|
|   111|        1|2199216|
|   101|        0| 864249|
|   118|        0|2142230|
|   111|        0|3062062|
|   120|        0|2208103|
|   108|        0|1239200|
|   104|        1|4636209|
+------+---------+-------+
only showing top 20 rows



In [ ]:
Users_4g_data_00=Users_4g_data.groupBy('circle').pivot('happiness').agg(psf.max("count").alias("4G_Users"))

In [ ]:
Users_4g_data_00=Users_4g_data_00.orderBy("circle")

In [ ]:
Users_4g_data_00.printSchema()

root
 |-- circle: integer (nullable = true)
 |-- 0: long (nullable = true)
 |-- 1: long (nullable = true)



In [ ]:
Users_4g_data_00=Users_4g_data_00.withColumnRenamed("0","Unhappy_4G_user")

In [ ]:
Users_4g_data_00=Users_4g_data_00.withColumnRenamed("1","Happy_4G_user")

In [ ]:
Users_4g_data_00.dtypes

[('circle', 'int'), ('Unhappy_4G_user', 'bigint'), ('Happy_4G_user', 'bigint')]

In [ ]:
Users_4g_data_00=Users_4g_data_00.withColumn("sum",Users_4g_data_00.Unhappy_4G_user+Users_4g_data_00.Happy_4G_user)

In [ ]:
Users_4g_data_00.show()

+------+---------------+-------------+--------+
|circle|Unhappy_4G_user|Happy_4G_user|     sum|
+------+---------------+-------------+--------+
|   101|         864249|       578810| 1443059|
|   102|        4561113|      5181373| 9742486|
|   103|        3155942|      2777044| 5932986|
|   104|        7719671|      4636209|12355880|
|   105|        7083169|      6441014|13524183|
|   106|        3347071|      2283982| 5631053|
|   107|        2849162|      2427471| 5276633|
|   108|        1239200|       870751| 2109951|
|   109|        2844770|      2638923| 5483693|
|   110|        3611373|      1917198| 5528571|
|   111|        3062062|      2199216| 5261278|
|   112|        4919013|      2698799| 7617812|
|   113|        1712891|      1299128| 3012019|
|   114|        3500147|      3132826| 6632973|
|   115|        1039510|      1717677| 2757187|
|   116|        5677936|      3034744| 8712680|
|   117|        1448183|       976733| 2424916|
|   118|        2142230|      3149410| 5

In [ ]:
Users_4g_data_00=Users_4g_data_00.withColumn("percentage_unhappy_4g_user",((Users_4g_data_00.Unhappy_4G_user)/(Users_4g_data_00.sum)*100))

In [ ]:
Users_4g_data_00.show()

+------+---------------+-------------+--------+--------------------------+
|circle|Unhappy_4G_user|Happy_4G_user|     sum|percentage_unhappy_4g_user|
+------+---------------+-------------+--------+--------------------------+
|   101|         864249|       578810| 1443059|        59.890066864902956|
|   102|        4561113|      5181373| 9742486|         46.81672624420502|
|   103|        3155942|      2777044| 5932986|         53.19314759886505|
|   104|        7719671|      4636209|12355880|        62.477711016940916|
|   105|        7083169|      6441014|13524183|        52.374099049088585|
|   106|        3347071|      2283982| 5631053|         59.43952223500649|
|   107|        2849162|      2427471| 5276633|         53.99583408586498|
|   108|        1239200|       870751| 2109951|         58.73122172031483|
|   109|        2844770|      2638923| 5483693|         51.87690120508205|
|   110|        3611373|      1917198| 5528571|         65.32199731178274|
|   111|        3062062| 

In [ ]:
Users_4g_data_00=Users_4g_data_00.withColumn("percentage_happy_4g_user",((Users_4g_data_00.Happy_4G_user)/(Users_4g_data_00.sum)*100))

In [ ]:
Users_4g_percnt=Users_4g_data_00.select("circle","percentage_unhappy_4g_user","percentage_happy_4g_user")

In [ ]:
Users_4g_percnt.show()

+------+--------------------------+------------------------+
|circle|percentage_unhappy_4g_user|percentage_happy_4g_user|
+------+--------------------------+------------------------+
|   101|        59.890066864902956|      40.109933135097044|
|   102|         46.81672624420502|       53.18327375579498|
|   103|         53.19314759886505|       46.80685240113494|
|   104|        62.477711016940916|       37.52228898305908|
|   105|        52.374099049088585|      47.625900950911415|
|   106|         59.43952223500649|       40.56047776499351|
|   107|         53.99583408586498|       46.00416591413502|
|   108|         58.73122172031483|       41.26877827968517|
|   109|         51.87690120508205|       48.12309879491795|
|   110|         65.32199731178274|       34.67800268821726|
|   111|        58.199965863807236|      41.800034136192764|
|   112|         64.57251767305362|      35.427482326946375|
|   113|        56.868532369815725|      43.131467630184275|
|   114|         52.7689

In [ ]:
Users_4g_percnt.toPandas().to_csv("Percentage_4g_users.csv",index=False,header=True)

In [ ]:
AON_data=AON_AOD_data.groupBy("circle","happiness").agg(
    udf_median(psf.collect_list(psf.col('aon'))).alias("Median_AON"))

In [ ]:
AOD_data=AON_AOD_data.groupBy("circle","happiness").agg(
    udf_median(psf.collect_list(psf.col('aon_curr_handset'))).alias("Median_AOD"))

In [ ]:
AON_data=AON_data.orderBy("circle")

In [ ]:
AOD_data=AOD_data.orderBy("circle")

In [ ]:
AON_data_00=AON_data.groupBy('circle').pivot('happiness').agg(psf.max('Median_AON').alias('Median_AON'))

In [ ]:
AOD_data_00=AOD_data.groupBy('circle').pivot('happiness').agg(psf.max('Median_AOD').alias('Median_AOD'))

In [ ]:
AON_data_00=AON_data_00.withColumnRenamed("0","Unhappy_customer_AON")

In [ ]:
AON_data_00=AON_data_00.withColumnRenamed("1","Happy_customer_AON")

In [ ]:
AOD_data_00=AOD_data_00.withColumnRenamed("0","Unhappy_customer_AOD")

In [ ]:
AOD_data_00=AOD_data_00.withColumnRenamed("1","Happy_customer_AOD")

In [ ]:
AON_data_00.toPandas().to_csv("AON_customer_happiness_wise.csv",index=False,header=True)

In [ ]:
AOD_data_00.toPandas().to_csv("AOD_customer_happiness_wise.csv",index=False,header=True)

In [ ]:
#ARPA_table_6.show()

In [ ]:
#ARPA_table_6_final=ARPA_table_6.groupBy("circle").count()

In [ ]:
REC_data=spark.sql("select * from edw_mart.fact_unified_rec_base_dly where event_date='2019-03-15'")

In [ ]:
REC_data.printSchema()

root
 |-- subscriber_id: decimal(25,0) (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- report_date: date (nullable = true)
 |-- last_usage_date: date (nullable = true)
 |-- circle_id: short (nullable = true)
 |-- service_type: byte (nullable = true)
 |-- voice_network: short (nullable = true)
 |-- voice_og_network: short (nullable = true)
 |-- data_network: short (nullable = true)
 |-- video_network: short (nullable = true)
 |-- overall_network: short (nullable = true)
 |-- sms_og_network: short (nullable = true)
 |-- load_date: timestamp (nullable = true)
 |-- load_user: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- event_date: string (nullable = true)
 |-- circle: string (nullable = true)



In [ ]:
REC=REC_data.select("msisdn","overall_network","product_type")

In [ ]:
REC_users=CHI_data.join(REC,on="msisdn",how="left")

In [ ]:
REC_users.select("product_type").distinct().show()

+------------+
|product_type|
+------------+
|      OTHERS|
|     NON_M2M|
|        null|
|         M2M|
|      BUNDLE|
|       SMART|
+------------+



In [ ]:
bundle_users=REC_users.where("product_type == 'BUNDLE'")

In [ ]:
bundle_data=bundle_users.groupBy("circle","happiness").count()

In [ ]:
Bundle_data_00=bundle_data.groupBy('circle').pivot('happiness').agg(psf.max("count").alias("bundle_Users"))

In [ ]:
Bundle_data_00=Bundle_data_00.orderBy("circle")

In [ ]:
Bundle_data_00.show()

+------+-------+-------+
|circle|      0|      1|
+------+-------+-------+
|   101| 660498| 772350|
|   102|2462047|3763624|
|   103|2243954|2611110|
|   104|5720350|5241658|
|   105|5633489|6694922|
|   106|2367641|2694693|
|   107|1798751|2647112|
|   108| 704533| 944299|
|   109|1340835|1974010|
|   110|2192416|2558528|
|   111|2043681|2168593|
|   112|2692219|2922103|
|   113| 843390|1296192|
|   114|2733742|3750501|
|   115| 817818|1094380|
|   116|5492515|4839330|
|   117|1132330|1296739|
|   118|2716417|2641718|
|   119|6277455|5938714|
|   120|2424082|2036647|
+------+-------+-------+
only showing top 20 rows



In [ ]:
Bundle_data_00=Bundle_data_00.withColumnRenamed("0","unhappy_bundle_user")

In [ ]:
Bundle_data_00=Bundle_data_00.withColumnRenamed("1","happy_bundle_user")

In [ ]:
Bundle_data_00=Bundle_data_00.withColumn("sum",Bundle_data_00.unhappy_bundle_user + Bundle_data_00.happy_bundle_user)

In [ ]:
Bundle_data_00=Bundle_data_00.withColumn("percent_users_bundle_unhappy",
                                         ((Bundle_data_00.unhappy_bundle_user)/(Bundle_data_00.sum)*100))

In [ ]:
Bundle_data_00=Bundle_data_00.withColumn("percent_users_bundle_happy",
                                         ((Bundle_data_00.happy_bundle_user)/(Bundle_data_00.sum)*100))

In [ ]:
Bundle_data_00.show()

+------+-------------------+-----------------+--------+----------------------------+--------------------------+
|circle|unhappy_bundle_user|happy_bundle_user|     sum|percent_users_bundle_unhappy|percent_users_bundle_happy|
+------+-------------------+-----------------+--------+----------------------------+--------------------------+
|   101|             660498|           772350| 1432848|          46.096864426652374|        53.903135573347626|
|   102|            2462047|          3763624| 6225671|          39.546693039192085|        60.453306960807915|
|   103|            2243954|          2611110| 4855064|          46.218834602386295|        53.781165397613705|
|   104|            5720350|          5241658|10962008|          52.183413841697615|        47.816586158302385|
|   105|            5633489|          6694922|12328411|           45.69517515274272|         54.30482484725728|
|   106|            2367641|          2694693| 5062334|           46.76975086985568|        53.230249130

In [ ]:
On_bundle_users=Bundle_data_00.select("circle","percent_users_bundle_unhappy","percent_users_bundle_happy")

In [ ]:
On_bundle_users.toPandas().to_csv("bundle_users_happiness_wise.csv",index=False,header=True)

In [ ]:
total_rec_non_rec=REC_users.groupBy("circle","happiness").count()

In [ ]:
total_rec_non_rec=total_rec_non_rec.orderBy("circle")

In [ ]:
total_rec_non_rec_data=total_rec_non_rec.groupBy('circle').pivot('happiness').agg(psf.max("count"))

In [ ]:
REC_users.select("overall_network").distinct().show()

+---------------+
|overall_network|
+---------------+
|            -99|
|           null|
|              1|
|              3|
|              4|
|              2|
+---------------+



In [ ]:
REC_users.select("service_type").distinct().show()

+------------+
|service_type|
+------------+
|        null|
|           1|
|           9|
+------------+



In [ ]:
Non_rec_users=REC_users.where("overall_network = -99")

In [ ]:
Non_rec_users.show()

+----------+---------+-----------+------+--------------------+-----------+---------------+---------+------------+-------------+----------------+------------+-------------+---------------+--------------+-------------------+---------+------------+----------+------+
|    msisdn|happiness|probability|circle|       subscriber_id|report_date|last_usage_date|circle_id|service_type|voice_network|voice_og_network|data_network|video_network|overall_network|sms_og_network|          load_date|load_user|product_type|event_date|circle|
+----------+---------+-----------+------+--------------------+-----------+---------------+---------+------------+-------------+----------------+------------+-------------+---------------+--------------+-------------------+---------+------------+----------+------+
|6200324977|        0|       0.58|   119|11901000006200324...| 2019-03-15|           null|      119|           9|          -99|             -99|         -99|          -99|            -99|           -99|2019-0

In [ ]:
Non_rec=Non_rec_users.groupBy("circle","happiness").count()

In [ ]:
Non_REC_data=Non_rec.groupBy('circle').pivot('happiness').agg(psf.max("count").alias("Non_REC_Users"))

In [ ]:
Non_REC_data.show()

+------+--------+-------+
|circle|       0|      1|
+------+--------+-------+
|   108| 1704764| 130453|
|   101| 1067183| 117436|
|   115|  207105|1605258|
|   103| 1794960| 322157|
|   122| 3143569| 373456|
|   111| 5509086|1266822|
|   120| 3856233| 729176|
|   117| 1464278|  97750|
|   112|10824776| 630376|
|   107| 2838325| 237170|
|   114| 3109718| 883964|
|   102| 3066067| 842555|
|   113| 2336898| 239696|
|   121|11786241| 777738|
|   109| 2021303| 295555|
|   105| 5729148| 644574|
|   110|10055066| 490615|
|   106| 5582458|1049506|
|   116|17118769|1094380|
|   123| 2579729| 266850|
+------+--------+-------+
only showing top 20 rows



In [ ]:
Non_REC_data=Non_REC_data.orderBy("circle")

In [ ]:
Non_REC_data=Non_REC_data.withColumnRenamed("0","unhappy_customer_Non_rec")

In [ ]:
Non_REC_data=Non_REC_data.withColumnRenamed("1","happy_customer_Non_rec")

In [ ]:
total_rec_non_rec_data.show()

+------+--------+--------+
|circle|       0|       1|
+------+--------+--------+
|   101| 2758872| 1401007|
|   102| 9781393| 7822140|
|   103| 7296947| 4664121|
|   104|26251196|10199069|
|   105|18729856|11173957|
|   106|12381192| 6307594|
|   107| 7432150| 4384834|
|   108| 3579736| 1626906|
|   109| 6334935| 4040504|
|   110|15941765| 4812810|
|   111|10545915| 5279472|
|   112|17824852| 5949578|
|   113| 4880128| 2415227|
|   114| 9700379| 7046274|
|   115| 2828945| 4644479|
|   116|33073653|10279293|
|   117| 4169508| 2176323|
|   118| 8821189|13037923|
|   119|36180348|13578122|
|   120|10310613| 4730032|
+------+--------+--------+
only showing top 20 rows



In [ ]:
NonREC_precent_data=Non_REC_data.join(total_rec_non_rec_data,on="circle")

In [ ]:
NonREC_precent_data.show()

+------+------------------------+----------------------+--------+--------+
|circle|unhappy_customer_Non_rec|happy_customer_Non_rec|       0|       1|
+------+------------------------+----------------------+--------+--------+
|   108|                 1704764|                130453| 3579736| 1626906|
|   101|                 1067183|                117436| 2758872| 1401007|
|   115|                  207105|               1605258| 2828945| 4644479|
|   103|                 1794960|                322157| 7296947| 4664121|
|   122|                 3143569|                373456| 7145309| 3391501|
|   111|                 5509086|               1266822|10545915| 5279472|
|   120|                 3856233|                729176|10310613| 4730032|
|   117|                 1464278|                 97750| 4169508| 2176323|
|   112|                10824776|                630376|17824852| 5949578|
|   107|                 2838325|                237170| 7432150| 4384834|
|   114|                 

In [ ]:
NonREC_precent_data=NonREC_precent_data.withColumnRenamed("0","total_unhappy")

In [ ]:
NonREC_precent_data=NonREC_precent_data.withColumnRenamed("1","total_happy")

In [ ]:
NonREC_precent_data=NonREC_precent_data.withColumn("%non_rec_unhappy_cust",
                               ((NonREC_precent_data.unhappy_customer_Non_rec)/(NonREC_precent_data.total_unhappy)*100))

In [ ]:
NonREC_precent_data=NonREC_precent_data.withColumn("%non_rec_happy_cust",
                               ((NonREC_precent_data.happy_customer_Non_rec)/(NonREC_precent_data.total_happy)*100))

In [ ]:
NonREC_precent_data.printSchema()

root
 |-- circle: integer (nullable = true)
 |-- unhappy_customer_Non_rec: long (nullable = true)
 |-- happy_customer_Non_rec: long (nullable = true)
 |-- total_unhappy: long (nullable = true)
 |-- total_happy: long (nullable = true)
 |-- %non_rec_unhappy_cust: double (nullable = true)
 |-- %non_rec_happy_cust: double (nullable = true)



In [ ]:
NonREC_precent=NonREC_precent_data.select("circle","%non_rec_unhappy_cust","%non_rec_happy_cust")

In [ ]:
NonREC_precent.show()

+------+---------------------+-------------------+
|circle|%non_rec_unhappy_cust|%non_rec_happy_cust|
+------+---------------------+-------------------+
|   108|   47.622617980767295|  8.018471872376153|
|   101|    38.68185983256925|  8.382256476948365|
|   115|    7.320927059380794|  34.56271413865796|
|   103|    24.59878083258656|  6.907132126289177|
|   122|    43.99486432287253| 11.011525575254142|
|   111|    52.23905180347083|  23.99524043313422|
|   120|   37.400618178569985| 15.415878793209009|
|   117|    35.11872383983914|  4.491520789882752|
|   112|    60.72856032689641| 10.595306087255265|
|   107|    38.18982394058247|  5.408870666483612|
|   114|   32.057695890026565| 12.545126686813484|
|   102|   31.345913613735792|  10.77141293814736|
|   113|   47.885998072181714|  9.924367357602412|
|   121|    51.69795083180635|  9.404507437444702|
|   109|   31.907241352910486|  7.314805281717331|
|   105|    30.58831845797426|  5.768538396917046|
|   110|   63.073731170921164| 

In [ ]:
NonREC_precent=NonREC_precent.orderBy("circle")

In [ ]:
NonREC_precent.toPandas().to_csv("Non_rec_happiness_wise.csv",index=False,header=True)